# Scraper to scrape Comments from subreddits related to mental health

In [1]:
import pymysql

from streamlit_app.db_config import MYSQL_DATABASE,MYSQL_HOST,MYSQL_PASSWORD,MYSQL_USER
try:
    # Establish connection
    mydb = pymysql.connect(
        host=MYSQL_HOST,
        user=MYSQL_USER,
        password=MYSQL_PASSWORD,
        database=MYSQL_DATABASE
    )

    mycursor = mydb.cursor()
    
    # Verify connection
    mycursor.execute("SELECT DATABASE()")
    current_db = mycursor.fetchone()
    print(f"✅ Connected to Database: {current_db[0]}")

except pymysql.MySQLError as err:
    print("❌ Error:", err)

✅ Connected to Database: mentalhealthdb


In [2]:
mycursor.execute("SHOW TABLES")
tables = mycursor.fetchall()
print("Existing tables:", tables)


Existing tables: (('reddit_comments',), ('reddit_posts',))


In [ ]:
import praw
import pymysql
from streamlit_app.db_config import MYSQL_HOST, MYSQL_USER, MYSQL_PASSWORD, MYSQL_DATABASE
from dotenv import load_dotenv
import os
import time

# Load environment variables
load_dotenv()
client_id = os.getenv("reddit_client_id")
client_secret = os.getenv("reddit_client_secret")
user_agent = os.getenv("reddit_user_agent")


# Initialize Reddit API

In [6]:

reddit = praw.Reddit(
    client_id=client_id,
    client_secret=client_secret,
    user_agent=user_agent
)

# Connect to MySQL

In [8]:

def connect_db():
    """Establishes a connection to the MySQL database."""
    try:
        conn = pymysql.connect(
            host=MYSQL_HOST,
            user=MYSQL_USER,
            password=MYSQL_PASSWORD,
            database=MYSQL_DATABASE,
            charset='utf8mb4',
            cursorclass=pymysql.cursors.DictCursor
        )
        return conn
    except pymysql.MySQLError as e:
        print("❌ Database Connection Error:", e)
        return None

# Scraper and Storage

In [4]:
def scrape_posts(subreddit_name="suicide", target=10000):
    """Scrapes posts from a subreddit across multiple listing types."""
    subreddit = reddit.subreddit(subreddit_name)
    posts = set()  # Use a set to avoid duplicates by post_id
    listing_types = [
        ("new", subreddit.new),
        ("hot", subreddit.hot),
        ("top_all", lambda limit, params: subreddit.top(time_filter="all", limit=limit, params=params)),
        ("top_year", lambda limit, params: subreddit.top(time_filter="year", limit=limit, params=params))
    ]

    for listing_name, listing_func in listing_types:
        if len(posts) >= target:
            break
        print(f"📌 Scraping {listing_name} posts...")
        after = None
        while len(posts) < target:
            try:
                # Fetch posts (limit=100 per request due to API constraints)
                new_posts = listing_func(limit=100, params={'after': after})
                new_posts_list = list(new_posts)

                if not new_posts_list:
                    print(f"📌 No more posts in {listing_name}")
                    break

                for post in new_posts_list:
                    post_data = {
                        "post_id": post.id,
                        "title": post.title,
                        "body": post.selftext if post.selftext else "No text",
                        "upvotes": post.score,
                        "created_at": post.created_utc  # UNIX timestamp
                    }
                    posts.add(post.id)  # Track unique post IDs
                    yield post_data  # Yield each post for immediate processing

                after = new_posts_list[-1].fullname
                print(f"📌 Scraped {len(posts)} unique posts so far from {listing_name}...")
                time.sleep(2)  # Respect Reddit's rate limit (30 requests/minute)

            except Exception as e:
                print(f"❌ Error scraping {listing_name}: {e}")
                break

    print(f"📌 Scraped a total of {len(posts)} unique posts from r/{subreddit_name}")

def store_posts(posts):
    """Inserts scraped posts into the MySQL database."""
    conn = connect_db()
    if not conn:
        return

    try:
        cursor = conn.cursor()
        count = 0

        query = """
        INSERT INTO reddit_posts (post_id, title, body, upvotes, created_at)
        VALUES (%s, %s, %s, %s, FROM_UNIXTIME(%s))
        ON DUPLICATE KEY UPDATE 
        title = VALUES(title), 
        body = VALUES(body), 
        upvotes = VALUES(upvotes), 
        created_at = VALUES(created_at)
        """

        for post in posts:
            cursor.execute(query, (
                post["post_id"], post["title"], post["body"], 
                post["upvotes"], post["created_at"]
            ))
            count += 1
            if count % 100 == 0:  # Commit in batches
                conn.commit()
                print(f"📌 Stored {count} posts so far...")

        conn.commit()
        print(f"✅ Stored {count} posts in MySQL!")

    except pymysql.MySQLError as e:
        print("❌ MySQL Error:", e)

    finally:
        cursor.close()
        conn.close()

# Run the scraping and storing process
if __name__ == "__main__":
    posts_generator = scrape_posts(subreddit_name="mentalhealth", target=10000)
    store_posts(posts_generator)

📌 Scraping new posts...
📌 Stored 100 posts so far...
📌 Scraped 100 unique posts so far from new...
📌 Stored 200 posts so far...
📌 Scraped 200 unique posts so far from new...
📌 Stored 300 posts so far...
📌 Scraped 300 unique posts so far from new...
📌 Stored 400 posts so far...
📌 Scraped 400 unique posts so far from new...
📌 Stored 500 posts so far...
📌 Scraped 500 unique posts so far from new...
📌 Stored 600 posts so far...
📌 Scraped 600 unique posts so far from new...
📌 Stored 700 posts so far...
📌 Scraped 700 unique posts so far from new...
📌 Stored 800 posts so far...
📌 Scraped 800 unique posts so far from new...
📌 Stored 900 posts so far...
📌 Scraped 900 unique posts so far from new...
📌 Scraped 975 unique posts so far from new...
📌 No more posts in new
📌 Scraping hot posts...
📌 Stored 1000 posts so far...
📌 Scraped 977 unique posts so far from hot...
📌 Stored 1100 posts so far...
📌 Scraped 977 unique posts so far from hot...
📌 Stored 1200 posts so far...
📌 Scraped 977 unique posts